In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src')

In [3]:
from pathlib import Path
import unidecode
import pandas as pd

import numpy as np
import re
from tqdm.notebook import tqdm
tqdm.pandas()

from utils import BelgeSayisiExtractor, MaddeSayisiExtractor, MevzuatNoExtractor

In [4]:
pd.options.display.max_columns = 20
pd.options.display.max_colwidth = 250

In [5]:
df = pd.read_csv("../data/kanunum-nlp-doc-analysis-dataset.csv")

In [6]:
df.shape

(4142, 16)

### Alanlar
- **id**                    : İçerik kimliği
- **kategori**              : İçerik Türü/Kategorisi
- **baslik**                : İçerik Başlığı
- **rega_no**               : İçeriğin yayınlandığı Resmi Gazete Sayısı
- **mukerrer_no**           : İçeriğin yayınlandığı Resmi Gazete Mükerrer Numarası (0 ise mükerrer olmadığını belirtir)
- **rega_tarihi**           : İçeriğin yayınlandığı Resmi Gazete Tarihi (`date:yyyy-mm-dd` formatında)
- **kurum**                 : İçeriği(Genelge, Cumhurbaşkanlığı(CB) Kararnamesi, Tüzük, Yönetmelik, Tebliğ) çıkaran kurum adı
- **mevzuat_no**            : İçeriğin ilgili numarası (Detaylar için ilgili kategoriye bakınız) 
- **belge_sayi**            : Genelge üzerindeki Sayı ifadesinin değeri (sadece işlenebilenler)
- **mevzuat_tarihi**        : İçeriğin ilgili Tarihi (`date:yyyy-mm-dd` formatında)
- **donem**                 : Komisyon Raporunun dönemini (örn: "27. Dönem")
- **sira_no**               : Komisyon Raporunun Sıra Sayısı (örn: "230", "42 ek 1" vb.)
- **madde_sayisi**          : İçeriğin(Kanun, KHK, CB Kararnamesi, Tüzük, Yönetmelik) bölümlerini (Madde sayısını ve işlenemeyen hükümleri/maddeleri, _madde metinlerinin_ içindeki "Madde <sayi> - ..." ifadeleri hariç )
- **data_text**             : İçerik metni (Büyük/Küçük Harf, Noktalama ve "\n" içermektedir)
- **url**                   : İçerik metninin esas alındığı websayfasının linki (kaynak linkler güncel olmayabilir)
- **kanunum_url**           : İçeriğin Kanunum linki

**İçerik Başlıkları**, Kanun, Resmi Gazete, Kanun Hükmünde Kararname, Cumhurbaşkanlığı Kararnamesi, Tebliğ, Yönetmelik ve Tebliğ için içeriğin adı olan başlık kullanılırken Komisyon Raporu, Genelge ve Özelgelerde tanımlayıcı olmak üzere işelenerek üretilmiş bir başlık tercih edilmiştir.

**Kurum Adı**, Kanunum veritabanına işlenmiş hali ile kullanılmaktadır. Örneğin; Üniversite Yönetmeliklerinde kurum adı ayrı ayrı üniversite adı kullanmak yerine tek bir ad ("Üniversite") olarak işlenmiştir. Kurum tespiti yapılırken Bakanlık isimlerinin değişmiş olabileceği ayrıca dikkate alınmalıdır.

## Kategoriler
 Kanun, Resmi Gazete, Kanun Hükmünde Kararname, Cumhurbaşkanlığı Kararnamesi, Genelge, Tebliğ, Yönetmelik, Komisyon Raporu, Tüzük ve Özelge olmak üzere toplam 10 kategori bulunmaktadır.

Tüm içerikler için `kategori`, `baslik`, `data_text`, `url`, `kanunum_url` alanları sunulmuş olup `url` alanı içerik metninin esas alındığı sayfa linkini belirtmektedir. Diğer dikkate alınacak alanlar aşağıdaki gibi olup detaylar alt başlıklarda verilmiştir.


| Kategori                                                               | İçerik Sayısı     | İlgili alanlar (Üst Veriler)                                                                                              |
| :-------                                                               | :------------     | :---------------------------                                                                                              |
| [Kanun](#kategori-kanun)                                               | 514               | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, mevzuat_no, mevzuat_tarihi, madde_sayisi, data_text, url             |
| [Kanun Hükmünde Kararname](#kategori-kanun-hükmünde-kararname-khk)     | 91                | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, mevzuat_no, mevzuat_tarihi, madde_sayisi, data_text, url             |
| [Resmi Gazete](#kategori-resmi-gazete)                                 | 546               | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, data_text, url                                                       |
| [Komisyon Raporu](#kategori-komisyon-raporu)                           | 500               | kategori, baslik, donem, sira_no, data_text, url                                                                          |
| [Genelge](#kategori-genelge)                                           | 385               | kategori, baslik, kurum, mevzuat_no, belge_sayi, mevzuat_tarihi, data_text, url                                           |
| [Cumhurbaşkanlığı Kararnamesi](#kategori-cumhurbaşkanlığı-kararnamesi) | 54                | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, kurum, mevzuat_no, mevzuat_tarihi, madde_sayisi, data_text, url      |
| [Tüzük](#kategori-tüzük)                                               | 82                | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, kurum, mevzuat_no, mevzuat_tarihi, madde_sayisi, data_text, url      |
| [Yönetmelik](#kategori-yönetmelik)                                     | 597               | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, kurum, mevzuat_no, mevzuat_tarihi, madde_sayisi, data_text, url      |
| [Tebliğ](#kategori-tebliğ)                                             | 623               | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, kurum, data_text, url                                                |
| [Özelge](#kategori-özelge)                                             | 750               | kategori, baslik, kurum, mevzuat_tarihi, data_text, url M

### BELGE SAYISI

In [7]:
belge_sayisi_extractor = BelgeSayisiExtractor()

In [8]:
belge_sayisi_extractor.extract(df)

'belge_sayi' cikariliyor... -> 'belge_sayi_pred'


  0%|          | 0/385 [00:00<?, ?it/s]

In [9]:
df[~df.belge_sayi.isna()][["belge_sayi", "belge_sayi_pred"]]

,belge_sayi,belge_sayi_pred
1025,B.07.0.GEL.0.44/4412-1-20248,B.07.0.GEL.0.44/4412-1-20248
1026,B.07.1.GİB.0.06.55-130[5511- 2801]-63856,B.07.1.GİB.0.06.55-130[5511- 2801]-63856
1027,B.07.0.GEL.0.71/7103-275 /12360,B.07.0.GEL.0.71/7103-275 /12360
1028,B.07.0.GEL.0.55-5511-2125/3599,B.07.0.GEL.0.55-5511-2125/3599
1029,B.07.0.GEL.0.38/3802- 57/037774,B.07.0.GEL.0.38/3802- 57/037774
1030,B.07.0.GEL.0.28/2810-13-866/02159,B.07.0.GEL.0.28/2810-13-866/02159
1031,B.07.0.GEL.0.38/3802-43 / 001089,B.07.0.GEL.0.38/3802-43 / 001089
1032,B.07.0.GEL.0.28/2810-13-866-068334,B.07.0.GEL.0.28/2810-13-866-068334
1033,B.07.0.GEL.0.71/7103-140,B.07.0.GEL.0.71/7103-140
1034,VUK.2/24491-212/07828,VUK.2/24491-212/07828


### MADDE SAYISI

In [10]:
madde_sayisi_extractor = MaddeSayisiExtractor()

In [11]:
madde_sayisi_extractor.extract(df)

'madde_sayisi' cikariliyor... -> 'madde_sayisi_pred'


  0%|          | 0/1338 [00:00<?, ?it/s]

In [12]:
df[~df.madde_sayisi.isna()][["madde_sayisi", "madde_sayisi_pred"]][:10]

,madde_sayisi,madde_sayisi_pred
0,19,19
1,56,56
2,149,149
3,11,11
4,21,21
5,12,12
6,76,88
7,15,15
8,17,94
9,41,41


### MEVZUAT NO

In [13]:
mevzuat_no_extractor = MevzuatNoExtractor()

In [14]:
mevzuat_no_extractor.extract(df)

'mevzuat_no' cikariliyor... -> 'mevzuat_no_pred'


  0%|          | 0/1723 [00:00<?, ?it/s]

In [15]:
df[df.kategori=="Genelge"].url.str[:25].unique()

array(['http://www.gib.gov.tr/nod', 'https://kms.kaysis.gov.tr',
       'https://www.bddk.org.tr/M', 'https://www.mevzuat.gov.t',
       'https://www.gib.gov.tr/no', 'http://www.sgk.gov.tr/wps',
       'http://www.tspakb.org.tr/', 'http://borsaistanbul.com/',
       'http://www.tkgm.gov.tr/tr', 'www.tkgm.gov.tr/tr/icerik',
       'http://www.mevzuat.gov.tr', 'https://www.resmigazete.g'],
      dtype=object)

In [16]:
df[(df.kategori=="Genelge") & (df.mevzuat_no != df.mevzuat_no_pred) & (df.url.str.contains("kms.kaysis.gov.tr"))]\
        [["mevzuat_no", "mevzuat_no_pred", "url", "baslik", "data_text"]][:20]

,mevzuat_no,mevzuat_no_pred,url,baslik,data_text
922,2006/16,16/375ek,https://kms.kaysis.gov.tr/Home/Goster/106310,"SOSYAL GÜVENLİK KURUMU, YATIRIMLARIN VE İSTİHDAMIN TEŞVİKİNE İLİŞKİN 26.06.2006 TARİHLİ 2006-16 SAYILI GENELGE",T.C.\nSOSYAL GÜVENLİK KURUMU BAŞKANLIĞI \n\nSosyal Sigortalar Kurumu Başkanlığı (Devredilen)\nSigorta İşleri Genel Müdürlüğü Sigorta Primleri Daire Başkanlığı\n\nSAYI : B.13.2.SSK.5.01.08.00/VIII-031-502522 ANKARA\nKONU : Yatırımların ve istihdam...
923,2005/16-358,16/358ek,https://kms.kaysis.gov.tr/Home/Goster/128284,"SOSYAL GÜVENLİK KURUMU, İHALE KONUSU İŞLERDEN KAYNAKLANAN BORÇLARIN TAKİBİ HAKKINDA 06/12/2005 TARİHLİ 2005/16-358 EK SAYILI GENELGE",T.C.\nSOSYAL SİGORTALAR KURUMU BAŞKANLIĞI \n\nSigorta İşleri Genel Müdürlüğü Sigorta Primleri Daire Başkanlığı\n\nSAYI : B. 13.2.SSK.5.01.08.00/VIII-031-946961 ANKARA\nKONU : ihale Konusu işlerden Kaynaklanan 06/12/2005\n\nBorçların Takibi\n\nGEN...
1135,2017/12,2017u2,https://kms.kaysis.gov.tr/Home/Goster/104268,"SOSYAL GÜVENLİK KURUMU, FATURA BEDELLERİNİN ÖDENMESİ GENELGESİNİN YÜRÜRLÜKTEN KALDIRILMASINA DAİR 2017/12 SAYILI GENELGE",sosyal güve ylik koromu\n\nTC.\nSOSYAL GÜVENLİK KURUMU BAŞKANLIĞI \n\nGenel Sağlık Sigortası Genel Müdürlüğü\n\nSayı : 35837838/226 16/03/2017\nKonu : Fatura Bedellerinin Ödenmesi Genelgesinin \n\nYürürlükten Kaldırılmasına Dair Genelge\n\nGENELG...
1139,1046330,NaN,https://kms.kaysis.gov.tr/Home/Goster/105981,"SOSYAL GÜVENLİK KURUMU, 2017/1 SAYILI GENELGE",T.C.\nSOSYAL GÜVENLİK KURUMU BAŞKANLIĞI \n\nStrateji Geliştirme Başkanlığı\n\ni rSayı\nU\n\n32945953-010.06.01-E. 1046330 23/02/2017\nKonu-: Strateji Geliştirme Başkanlığı Daire Başkanlıktan \n\nve Alt Birimlerin Görev Tanımlan\ni\n\nİÇ GENELGE \...
1148,2016/17,kurumumuzkadroluvesozlesmeliavukatlaritarafindanhukukmusavirligineiletilen,https://kms.kaysis.gov.tr/Home/Goster/75464,"SOSYAL GÜVENLİK KURUMU, HUKUK SERVİSLERİNİN İHTİYAÇLARINA İLİŞKİN 2016/17 SAYILI GENELGE",îw**.s mmmst\n\nT.C.\nSOSYAL GÜVENLİK KURUMU BAŞKANLIĞI \n\nHukuk Müşavirliği\n\nSayı : 72044944/6590735/13272/4301209 04/08/2016\nKonu : Hukuk Servislerinin İhtiyaçları\n\nGENELGE\n\nKurumumuz kadrolu ve sözleşmeli avukatları tarafından Hukuk Mü...
1151,2012/2,2012/02,https://kms.kaysis.gov.tr/Home/Goster/74355,"SOSYAL GÜVENLİK KURUMU, 17/1/2012 TARİHLİ VE 2012/2 SAYILI YEŞİLKART DEVRİ VE GENEL SAĞLIK SİGORTASI TESCİL İŞLEMLERİ GENELGESİ",TC.\nSOSYAL GÜVENLİK KURUMU BAŞKANLIĞI \n\nSigorta Primleri Genel Müdürlüğü\n\nSayı : B.13.2.SGK.0.10.03.27/GSS-34 17/1/2012\nKonu : Yeşil kart devri ve genel sağlık \n\nsigortası tescil işlemleri\n\nGENELGE\n2012/2\n\n1- Genel açıklamalar\n\n551...
1152,2015/28,2015/2j3,https://kms.kaysis.gov.tr/Home/Goster/69818,"SOSYAL GÜVENLİK KURUMU, 2014/28 SAYILI GENELGE","TC\nsosyal güvenlik kurumu başkanliği\n\nSigorta Primleri Genel Müdürlüğü\n\nSayı : 82200845/01006/6654040 30/12/2015\nKonu : 2014/28 Sayılı Genelge\n\nGENELGE \n2015 / 2j3\n\n20/10/2014 tarihli ve 2014/28 sayılı Genelgenin ""Çakışan süreler içind..."
1153,2015/23,2015/00,https://kms.kaysis.gov.tr/Home/Goster/60371,"SOSYAL GÜVENLİK KURUMU, 17/9/2015 TARİH VE 2015/23 SAYILI GENELGE",T.C.\nSOSYAL GÜVENLİK KURUMU BAŞKANLIĞI \n\nEmeklilik Hizmetleri Genel Müdürlüğü\n\nSayı : 56766929 -113/ O -/?./0f2O15\nKonu : Çalışma Gücü Kaybı ile Meslekte Kazanma Gücü \n\nKayıp Oranı Tespit İşlemleri\n\nGENELGE \n2015 /\n\n10/09/2014 tarihl...
1154,2015/20,2015/2.0,https://kms.kaysis.gov.tr/Home/Goster/57954,"SOSYAL GÜVENLİK KURUMU, 2015/20 SAYILI GENELGE","f\nT.C.\n\nSOSYAL GÜVENLİK KURUMU BAŞKANLIĞI \n\nHizmet Sunumu Genel Müdürlüğü\n\nSayı : 90974675/601040201/3731223 14/07/2015\nKonu : Pozisyon Bazında Iş/Görev Tanımları\n\nGENELGE \n2015/2.0\n\nBilindiği üzere, Yönetim Kurulunun 27/11/2014 tari..."
1158,2015/11,2015/0h,https://kms.kaysis.gov.tr/Home/Goster/57268,"SOSYAL GÜVENLİK KURUMU, 2014/26 SAYILI GENELGEDE GSS SÜRELERİYLE İLİGİLİ YAPILAN DEĞİŞİKİLİKLER",T.C.\nSOSYAL GÜVENLİK KURUMU BAŞKANLIĞI \n\nSigorta Primleri Genel Müdü

In [17]:
df.loc[1025]

id                                                                                                                                                                                                                                                                        2024
kategori                                                                                                                                                                                                                                                               Genelge
baslik                                                                                                                                                                GELİR İDARESİ BAŞKANLIĞI, YOLCU VE YÜK TAŞIMACILIĞI FAALİYETİNDE GÖTÜRÜLÜK DERECESİNE İLİŞKİN GENEL YAZI
rega_no                                                                                                                                                                                    

In [18]:
print(mevzuat_no_df.iloc[700].data_text)

NameError: name 'mevzuat_no_df' is not defined